In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd 
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split

In [3]:
#Load the data into Pandas dataframe
df = pd.read_csv('../task2_data/task2_en_training.tsv', sep='\t')[['class', 'tweet']]
df['class'] = df['class'].apply(lambda x: int(x))
df.head()

class                                              tweet
0      0                depression hurts, cymbalta can help
1      0  @jessicama20045 right, but cipro can make thin...
2      0         @fibby1123 are you on paxil .. i need help
3      0  @redicine the lamotrigine and sjs just made ch...
4      0  have decided to skip my #humira shot today. my...

In [4]:
df = df.iloc[np.random.permutation(len(df))]
df.head()

class                                              tweet
14754      0  @thegirlwithms i wish she was coming closer to...
16751      0  initial combo of linagliptin and metformin com...
11624      0               @devvypaws flovent 110mcg and proair
5059       0  i have no idea who that post menopausal #proli...
756        0  trazodone is nasty as fuck when it touches you...

In [7]:
X = df['tweet']
Y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X, Y , test_size=0.2)

tfidf = TfidfVectorizer()
tfidf.fit(X_train)
X_train = tfidf.transform(X_train)
X_test = tfidf.transform(X_test)
    
# type(X_train), type(y_train)

# X_train = Train['tweet']
# y_train = Train['type']

# X_test = Test['tweet']
# y_test = Test['type']

# tfidf = TfidfVectorizer()
# tfidf.fit(X_train)

# X_train = tfidf.transform(X_train)
# X_test = tfidf.transform(X_test)


In [16]:
X_train[1]

<1x30454 sparse matrix of type '<class 'numpy.float64'>'
	with 14 stored elements in Compressed Sparse Row format>

In [16]:
SVM = SVC(kernel='linear')
SVM.fit(X_train,y_train)

predictions_SVM = SVM.predict(X_test)

In [17]:
round(f1_score(predictions_SVM, y_test, average='macro')*100, 3)

66.386

# SHUFFLE SPLIT + Cross Val.

In [8]:
from sklearn.model_selection import ShuffleSplit, cross_val_score, cross_val_predict
from scipy.sparse import hstack

data = df
data = data.iloc[np.random.permutation(len(df))]
data.head()

class                                              tweet
3034       0  any dealers know a supplier with trazodone in ...
14795      0  status epilepticus. icu tutorials. risk of ten...
8784       0  $bmy 11:08 am bristol-myers and pfizer: in a s...
8339       0  one month from today. i will be 29 years old. ...
14827      0  science time  ever notice that an experiment/d...

In [9]:
X = data['tweet']
Y = data['class']

X_train, X_test, y_train, y_test = train_test_split(X, Y , test_size=0.2)

tfidf = TfidfVectorizer()
tfidf.fit(X_train)
X_train = tfidf.transform(X_train)
X_test = tfidf.transform(X_test)
    
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(16435, 30421)
(4109, 30421)
(16435,)
(4109,)


In [11]:
SVM2 = SVC(kernel='linear', C=1) #.fit(X_train,y_train)
tols = [0.1, 0.01, 0.001, 0.0001]
best_score = 0
best_tol = 0
for tol in tols:
    SVM2 = SVC(kernel='linear', C=1, tol=tol)
    scores = cross_val_score(SVM2, X_train, y_train, cv=3, scoring='f1_macro')
    m = scores.mean()
    if m > best_score:
        best_score = m
        best_tol = tol
    print('tol =', tol, 'score =', scores.mean())
    
print('best_tol =', best_tol, 'best_score =', best_score)

tol = 0.1 score = 0.6170430023711727
tol = 0.01 score = 0.6184031167612315
tol = 0.001 score = 0.6184031167612315
tol = 0.0001 score = 0.6184031167612315
best_tol = 0.01 best_score = 0.6184031167612315


In [14]:
print(classification_report(y_test, predictions_SVM))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94      3724
           1       0.05      0.01      0.02       385

    accuracy                           0.89      4109
   macro avg       0.48      0.50      0.48      4109
weighted avg       0.83      0.89      0.85      4109

